# Interactive Visualization of Daytimes and Workingtimes in Europe

## Data

In [61]:
import pandas as pd
import geopandas as gpd

RELOAD_DATA = False
eu_data_path = 'datasets/saved/eu_gpd.geojson'
city_data_path = 'datasets/saved/city_data.csv'
sunrise_data = 'datasets/saved/sunrise_data.csv'
avg_country_data_path = 'datasets/saved/avg_country.csv'
eu_geo_tz_path = 'datasets/saved/eu_geo_tz.geojson'

### EU countries to capitals and GeoPandas data

In [62]:
from sun_data import get_sunrise_data_avgs_for_countries
from geo_utils import load_eu_countries_as_geopandas, get_eu_city_data, get_avg_country_data

In [63]:
if RELOAD_DATA:
    print('Reloading data from datasets/saved folder ...')
    eu_gpd = gpd.read_file(eu_data_path)
    top_city_data = pd.read_csv(city_data_path)
    sun_data = gpd.read_file(sunrise_data)
    avg_country_data = pd.read_csv(avg_country_data_path)
    eu_geo_tz = gpd.read_file(eu_geo_tz_path)
else:
    print('Generating data ...')
    top_city_data = get_eu_city_data(3)
    top_city_data.to_csv(city_data_path, index=False)

    eu_gpd = load_eu_countries_as_geopandas()
    eu_gpd.to_file(eu_data_path, driver='GeoJSON', index=False)

    avg_country_data = get_avg_country_data(top_city_data, eu_gpd)
    avg_country_data.to_csv(avg_country_data_path, index=False)

    sun_data = get_sunrise_data_avgs_for_countries(top_city_data)
    sun_data.to_csv(sunrise_data, index=False)

    eu_geo_tz = eu_gpd.merge(avg_country_data)
    eu_geo_tz = eu_geo_tz.merge(sun_data, left_on=['iso_a2', 'dst'], right_on=['country_ISO_A2', 'dst'])
    eu_geo_tz.to_file(eu_geo_tz_path, driver='GeoJSON', index=False)

Generating data ...


C:\Users\phili\PycharmProjects\CircadianRythmEU\geo_utils.py:28: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\phili\miniconda3\envs\circadian_rythm\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [65]:
eu_gpd.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,iso_a2
43,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-5750519.703 463075.975, -5816...",FR
110,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((1227561.058 8149359.996, 1276642.191...",SE
113,37970874.0,Europe,Poland,POL,595858,"POLYGON ((2614241.130 7153601.786, 2619073.310...",PL
114,8877067.0,Europe,Austria,AUT,445075,"POLYGON ((1890167.860 6127424.985, 1881717.299...",AT
115,9769949.0,Europe,Hungary,HUN,163469,"POLYGON ((2458558.676 6177394.292, 2520364.547...",HU


In [66]:
top_city_data.head()

,population,CODE,country_ISO_A2,NAME,longitude,latitude,mercantor_x,mercantor_y,social_timezone,utc_sun_timezone_offset,summertime_longitudinal_diff_km,longitudinal_diff_km
0,1766746,AT001C,AT,Wien,16.372504,48.208354,1.822579e+06,6.141588e+06,CET,1,1090.199664,-109.800336
1,269997,AT002C,AT,Graz,15.438279,47.070868,1.718581e+06,5.953649e+06,CET,1,1164.937712,-35.062288
2,193814,AT003C,AT,Linz,14.286198,48.305908,1.590332e+06,6.157899e+06,CET,1,1257.104160,57.104160
3,1226329,BE001C,BE,Bruxelles/Brussel,4.351697,50.846557,4.844287e+05,6.594196e+06,CET,1,2051.864240,851.864240
4,530627,BE002C,BE,Antwerpen,4.399708,51.221110,4.897733e+05,6.660499e+06,CET,1,2048.023352,848.023352


In [67]:
sun_data.head()

,country_ISO_A2,summer_period,winter_period,dst,summer_diff_h,winter_diff_h
0,AT,05:56,08:09,True,3.058951,0.835794
1,BE,06:31,09:00,True,2.471296,-0.015735
2,BG,06:26,08:17,True,2.555041,0.706898
3,CY,06:06,07:30,True,2.898457,1.483557
4,CZ,05:47,08:10,True,3.200257,0.821104


In [68]:
eu_geo_tz.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,iso_a2,social_timezone,utc_sun_timezone_offset,mean_longitudinal_diff_km,...,pop_norm,dst,weights,weighted_mean_longdiff,norm_weighted_mean_longdiff,country_ISO_A2,summer_period,winter_period,summer_diff_h,winter_diff_h
0,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-5750519.703 463075.975, -5816...",FR,CET,1,865.323856,...,0.805207,False,1.805207,696.765077,0.418977,FR,05:44,07:50,3.251337,1.166518
1,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-5750519.703 463075.975, -5816...",FR,CET,1,2065.323856,...,0.805207,True,1.805207,1663.013825,1.000000,FR,06:44,08:50,2.251337,0.166518
2,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((1227561.058 8149359.996, 1276642.191...",SE,CET,1,52.312875,...,0.117140,False,1.117140,6.127927,0.003685,SE,04:32,07:48,4.461780,1.200000
3,10285453.0,Europe,Sweden,SWE,530883,"POLYGON ((1227561.058 8149359.996, 1276642.191...",SE,CET,1,1252.312875,...,0.117140,True,1.117140,146.695862,0.088211,SE,05:32,08:48,3.461780,0.200000
4,37970874.0,Europe,Poland,POL,595858,"POLYGON ((2614241.130 7153601.786, 2619073.310...",PL,CET,1,-410.681931,...,0.452668,False,1.452668,-185.902704,0.111787,PL,04:29,07:01,4.515921,1.983296


### Generate top n cities per country with timezone features

## Interactive visualization with Panel/Bokeh

In [69]:
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, CategoricalColorMapper, ColorBar, ColumnDataSource, LabelSet, HoverTool
from bokeh.transform import factor_cmap
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Plasma256
from bokeh.palettes import brewer
import json
import panel as pn
from datetime import datetime, date
import numpy as np

data_field = 'social_timezone'
bokeh_tools = 'wheel_zoom, pan, box_zoom, reset, save'
colorbar_settings = {'title_text_font_size':'12pt','label_standoff':12}

def get_bokeh_geodata_source(gpd_df):
    json_data = json.dumps(json.loads(gpd_df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(data, winter_period_active: bool):
    p = figure(toolbar_location='right', tools=bokeh_tools, active_scroll ="wheel_zoom",
               title="Time difference between sunrise and 9:00 for EU countries",
               x_range=(-1.3*10**6, 4*10**6),
               y_range=(4*10**6,9*10**6))
    p.title.text_font_size = '20px'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # ADD MAP TILES ---------------------------------------------------------------------------
    p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

    # ADD TIMEZONE LINES

    # ADD GEO STUFF FOR COUNTRIES AS A WHOLE -------------------------------------------------
    geo_data_source = get_bokeh_geodata_source(data)

    values = data[data_field]
    palette = brewer['OrRd'][3]
    palette = palette[::-1]
    color_mapper = CategoricalColorMapper(palette = palette, factors=values.unique().tolist())
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title='Timezone', **colorbar_settings)
    country_tz = p.patches('xs','ys', source=geo_data_source,
            fill_color={'field': data_field, 'transform': color_mapper},
            line_color='blue',
            line_width=0.5,
            fill_alpha=0.8)
    p.add_layout(color_bar, 'below')

    # ===================================================================================================================

    time_diff_col = data['winter_diff_h' if winter_period_active else 'summer_diff_h']
    avg_sunrise = data['winter_period' if winter_period_active else 'summer_period']
    bar_color = time_diff_col.apply(lambda x: '#ff0000' if np.sign(x) < 0 else 'blue')
    #ADD BARS FOR DISTANCE TO EAST MERIDIAN EFFECT
    length_scale = 100000
    divider_len = 50000
    bar_data_source = ColumnDataSource(dict(
            x0=data['mercantor_x'],
            y0=data['mercantor_y'],
            x1=data['mercantor_x'] + (length_scale *  time_diff_col),
            y1=data['mercantor_y'],
            lwd=1 + data['pop_norm'] * 10,
            l_col=bar_color,
            avg_sunrise=avg_sunrise,
            time_diff=time_diff_col,
            country=data['name'],
            pop_weight=data['pop_norm'],
            long_diff=data['mean_longitudinal_diff_km'],
            weighted_long_diff=data['weighted_mean_longdiff'],
            long_diff_norm=data['norm_weighted_mean_longdiff'],
            text=data['name'],
            text_y=(data['mercantor_y'] + divider_len / 2) + 1000
        )
    )
    divider_data_source = ColumnDataSource(dict(
            x0=data['mercantor_x'],
            y0=data['mercantor_y'] - divider_len / 2,
            x1=data['mercantor_x'],
            y1=data['mercantor_y'] + divider_len / 2,
            l_col=bar_color
    ))
    longdiff_quads = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width="lwd", line_color='l_col', source=bar_data_source)
    londiff_diviers = p.segment(x0="x0", y0="y0", x1="x1", y1="y1", line_width=3, line_color='l_col', source=divider_data_source)
    longdiff_label = p.text(x="x0",y="text_y", text="text", source=bar_data_source)

    # TOOLTIPS FOR CITY DATA
    tooltips_longquads = [
        ('Country', '@country'),
        ('Avg. sunrise', '@avg_sunrise'),
        ('Time difference from sunrise to 9:00', '@time_diff'),
        ('Population weight', '@pop_weight'),
        ('Relative distance (km) to timezone border', '@long_diff'),
        ('Weighted (population size) diff. to east timezone border', '@weighted_long_diff'),
        ('Normalized, weighted (population size) effect of distance to east meridian', '@long_diff_norm')
    ]
    p.add_tools(HoverTool(renderers=[longdiff_label], tooltips=tooltips_longquads, name='long_quads'))
    return p

In [70]:
from bokeh.models import DataTable, TableColumn

def bokeh_country_table(country_data):
    country_data_sorted = country_data.sort_values('norm_weighted_mean_longdiff', ascending=False)
    source = ColumnDataSource(country_data_sorted)
    columns = [
        TableColumn(field='name', title='Country Name'),
        TableColumn(field="iso_a2",title="Country Code (ISO_A2)"),
        TableColumn(field="social_timezone", title="Social Timezone"),
        TableColumn(field="pop_est", title="Estimated population"),
        TableColumn(field="dst", title="DST active"),
        TableColumn(field="summer_period", title="Avg. sunrise in summer period"),
        TableColumn(field="winter_period", title="Avg. sunrise in winter period"),
        TableColumn(field="winter_diff_h", title="Difference between sunrise to 9:00 in winter"),
        TableColumn(field="summer_diff_h", title="Difference between sunrise to 9:00 in summer"),
        TableColumn(field="norm_weighted_mean_longdiff", title="Normalized weighted effect of dist. to east meridian"),
        TableColumn(field="mean_longitudinal_diff_km", title="Avg. dist. to east meridian (km)"),
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [71]:
def bokeh_sun_table(sun_data):
    source = ColumnDataSource(sun_data)

    # Add data table
    columns = [
        TableColumn(field="iso_a3", title="Country Code (ISO_A3)"),
        TableColumn(field="year", title="Year"),
        TableColumn(field="month", title="Month"),
        TableColumn(field="day", title="Day"),
        TableColumn(field="sunrise_UTC", title="Sunrise (UTC/GMT)"),
        TableColumn(field="sunset_UTC", title="Sunrise (UTC/GMT)")
    ]
    data_table = DataTable(source=source, columns=columns)
    return data_table

In [72]:
def map_visualization():
    # CREATE MAP  ----------------------------------------------------------------------------------
    # Create Map Panel
    map_pane = pn.pane.Bokeh(sizing_mode='scale_both', width_policy='max')
    dst_text = pn.widgets.StaticText(value='Daylight savings time (DST) enabled:')
    dst_toggle = pn.widgets.Switch(name="DST Toggle")

    period_text = pn.widgets.StaticText(value='Summer Period (Last Sunday in March) / Winter Period (Last Sunday in October):')
    period_toggle = pn.widgets.Switch(name="Summer/Winter period Toggle")

    def update_map(event):
        eu_geo_tz_filter = eu_geo_tz[eu_geo_tz['dst'] == dst_toggle.value]
        map_pane.object = bokeh_plot_map(eu_geo_tz_filter, period_toggle.value)
    dst_toggle.param.watch(update_map, 'value')
    dst_toggle.param.trigger('value')
    period_toggle.param.watch(update_map, 'value')
    period_toggle.param.trigger('value')

    # CREATE DATATABLES ----------------------------------------------------------------------------------
    sizing_dict = dict(sizing_mode='stretch_both', width_policy='auto', margin=10)
    # Create City Table Panel
    country_data_pane = pn.pane.Bokeh(**sizing_dict)
    country_data_pane.object = bokeh_country_table(eu_geo_tz.drop(columns=['geometry']))

    # Create panel application layout
    map_vis = pn.Column(pn.Column(pn.Row(dst_text, dst_toggle), pn.Row(period_text, period_toggle)), map_pane)
    tabs = pn.Tabs(('Map', map_vis), ('Country Data', country_data_pane))
    return tabs

app = map_visualization()

In [73]:
app.show()

Launching server at http://localhost:1675
